# 멀티 세션 Evaluation

이 노트북은 Strands Evals를 사용하여 agent 세션을 평가합니다. Strands Evals는 LLM을 심사자로 사용하는 확장 가능한 LLM 기반 evaluation 프레임워크입니다. 각 세션에 대해 AgentCore Observability에서 trace를 가져오고, evaluator를 실행하며, 대시보드 상관관계를 위해 원본 trace ID와 함께 결과를 다시 로깅합니다.

**이 노트북은 두 가지 evaluator를 시연합니다:**
- **OutputEvaluator**: 응답 품질 점수 (관련성, 정확성, 완전성)
- **TrajectoryEvaluator**: Tool 사용 점수 (선택, 효율성, 순서)

Strands Evals는 거의 모든 evaluation 유형에 대한 커스텀 evaluator를 지원합니다. 프레임워크의 강력함은 rubric 시스템에 있습니다—기준을 정의하면 LLM이 일관되게 적용합니다.

**워크플로우:**
1. discovery 노트북에서 세션 로드 (또는 커스텀 세션 ID 제공)
2. 각 세션에 대해: trace 가져오기, evaluation case 생성, evaluator 실행
3. EMF 형식으로 AgentCore에 결과 로깅
4. 요약 통계 생성

**전제 조건:** 먼저 세션 discovery 노트북을 실행하거나 세션 ID 목록을 준비하세요.

## 이 노트북의 위치

이것은 **Notebook 2 (Option A)** - 사용자가 정의한 커스텀 rubric을 사용하여 세션을 평가합니다.

![Notebook Workflow](images/notebook_workflow.svg)

## 데이터 흐름

Evaluation 파이프라인은 AgentCore Observability trace를 점수화된 결과로 변환합니다:

![Evaluation Pipeline](images/evaluation_pipeline.svg)

## 설정

Strands Evals evaluator와 AgentCore Observability 상호작용을 위한 유틸리티 클래스를 포함한 필수 모듈을 가져옵니다. 구성은 `config.py`에서 로드됩니다.

In [ ]:
import logging
import sys
from datetime import datetime, timedelta, timezone
from typing import List

# 현재 디렉토리를 path에 추가하여 로컬 모듈 import 가능하게 설정
sys.path.insert(0, ".")

from config import (
    AWS_REGION,
    AWS_ACCOUNT_ID,
    SOURCE_LOG_GROUP,
    EVAL_RESULTS_LOG_GROUP,
    LOOKBACK_HOURS,
    MAX_CASES_PER_SESSION,
    DISCOVERED_SESSIONS_PATH,
    RESULTS_JSON_PATH,
    EVALUATION_CONFIG_ID,
    setup_cloudwatch_environment,
)

from utils import (
    CloudWatchSessionMapper,  # CloudWatch span을 Strands Eval 형식으로 변환
    ObservabilityClient,  # AgentCore Observability에서 trace 데이터 가져오기
    SessionDiscoveryResult,
    SessionInfo,
    send_evaluation_to_cloudwatch,  # Evaluation 결과를 CloudWatch에 로깅
)

# Strands Evals: LLM 기반 evaluation 프레임워크
from strands_evals import Case, Experiment
from strands_evals.evaluators import OutputEvaluator, TrajectoryEvaluator
from strands_evals.types.trace import AgentInvocationSpan, ToolExecutionSpan

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

## 구성

CloudWatch 메트릭을 위한 evaluator 이름을 정의합니다. 이 이름들은 AgentCore Observability 대시보드에 표시되며 `Custom.YourEvaluatorName` 규칙을 따라야 합니다. `EVALUATION_CONFIG_ID`는 `config.py`에서 로드됩니다.

In [ ]:
# CloudWatch 메트릭에 표시될 evaluator 이름 (Custom. prefix 필수)
OUTPUT_EVALUATOR_NAME = "Custom.OutputEvaluator"  # 응답 품질 평가
TRAJECTORY_EVALUATOR_NAME = "Custom.TrajectoryEvaluator"  # Tool 사용 패턴 평가

## CloudWatch 환경

Evaluation 결과 로깅에 필요한 환경 변수를 구성합니다. OTEL 리소스 속성에 대해 `config.py`의 `SERVICE_NAME`을 사용합니다.

In [ ]:
setup_cloudwatch_environment()

## 세션 로드

Discovery 노트북 JSON 출력에서 세션을 로드합니다. 또는 `USE_JSON_FILE = False`로 설정하고 특정 세션의 타겟 재평가를 위해 커스텀 세션 ID를 직접 제공할 수 있습니다.

In [ ]:
# Set to False to provide custom session IDs instead
USE_JSON_FILE = True

if USE_JSON_FILE:
    discovery_result = SessionDiscoveryResult.load_from_json(DISCOVERED_SESSIONS_PATH)
    sessions_to_process = discovery_result.sessions
else:
    # Provide custom session IDs here
    session_ids = [
        "your-session-id-here",
    ]
    sessions_to_process = [
        SessionInfo(
            session_id=sid,
            span_count=0,
            first_seen=datetime.now(timezone.utc),
            last_seen=datetime.now(timezone.utc),
            discovery_method="user_provided",
        )
        for sid in session_ids
    ]

print(f"Loaded {len(sessions_to_process)} sessions")

## Evaluator Rubric

Rubric은 evaluation 기준을 정의합니다. Evaluator는 rubric과 agent의 출력을 LLM에 전송하고, LLM은 심사자 역할을 하여 설명과 함께 점수(0.0-1.0)를 반환합니다.

**효과적인 rubric 작성:**
- 좋은 품질과 나쁜 품질을 구성하는 요소에 대해 구체적으로 작성
- 점수 기준점 포함 (1.0 vs 0.5 vs 0.0은 무엇을 의미하는가?)
- Agent의 도메인과 관련된 측정 가능한 기준에 집중

아래에서 이러한 rubric을 커스터마이즈하세요. 기본 rubric은 일반적인 응답 품질과 tool 사용 패턴을 평가합니다.

In [ ]:
output_rubric = """
Evaluate the agent's response based on:
1. Relevance: Does the response directly address the user's question?
2. Accuracy: Is the information factually correct?
3. Completeness: Does the response provide sufficient detail?

Score 0.0-1.0: 1.0=excellent, 0.5=adequate, 0.0=poor
"""

trajectory_rubric = """
Evaluate the agent's tool usage based on:
1. Tool Selection: Did the agent choose appropriate tools?
2. Efficiency: Were tools used without unnecessary calls?
3. Logical Sequence: Were tools used in a logical order?

Score 0.0-1.0: 1.0=optimal, 0.5=acceptable, 0.0=poor
"""

## 헬퍼 함수

이 함수들은 AgentCore Observability trace와 Strands Evals를 연결합니다:

- `task_fn(case)`: OutputEvaluator가 rubric에 대해 점수를 매기기 위해 agent의 실제 응답을 반환합니다.

- `trajectory_task_fn(case)`: TrajectoryEvaluator가 tool 사용 패턴을 평가하기 위해 응답과 tool 시퀀스를 모두 반환합니다.

- `create_cases_from_session(session)`: Strands Eval Session을 evaluation Case로 변환합니다. AgentInvocationSpan에서 사용자 프롬프트를 추출하고, ToolExecutionSpan 객체에서 tool 이름을 추출하며, CloudWatch 상관관계를 위해 원본 trace_id를 보존합니다.

- `log_case_result_to_cloudwatch(case, ...)`: 원본 trace_id를 사용하여 AgentCore Observability에 evaluation 결과를 전송하여 대시보드에서 원본 trace와 함께 점수를 볼 수 있도록 합니다.

In [ ]:
def task_fn(case: Case) -> str:
    """Return actual output from trace metadata."""
    return (case.metadata.get("actual_output", ""))


def trajectory_task_fn(case: Case):
    """Return output and trajectory from trace metadata."""
    return {"output": case.metadata.get("actual_output", ""), "trajectory": case.metadata.get("trajectory_for_eval", [])}

def log_case_result_to_cloudwatch(case: Case, evaluator_name: str, score: float, explanation: str, label: str = None) -> bool:
    """Log evaluation result to CloudWatch with original trace ID."""
    trace_id = case.metadata.get("trace_id", "")
    if not trace_id:
        return False
    return send_evaluation_to_cloudwatch(
        trace_id=trace_id,
        session_id=case.session_id,
        evaluator_name=evaluator_name,
        score=score,
        explanation=explanation,
        label=label,
        config_id=EVALUATION_CONFIG_ID,
    )


def create_cases_from_session(session, session_id: str, max_cases: int = None) -> List[Case]:
    """Create evaluation cases from a Strands Eval Session."""
    cases = []
    for i, trace in enumerate(session.traces):
        if max_cases and len(cases) >= max_cases:
            break
        agent_span = None
        tool_names = []
        for span in trace.spans:
            if isinstance(span, AgentInvocationSpan):
                agent_span = span
            elif isinstance(span, ToolExecutionSpan):
                tool_names.append(span.tool_call.name)
        if agent_span:
            case = Case(
                name=f"trace_{i+1}_{trace.trace_id[:8]}",
                input=agent_span.user_prompt or "",
                expected_output="",
                session_id=session_id,
                metadata={
                    "actual_output": agent_span.agent_response or "",
                    "actual_trajectory": tool_names,
                    "trace_id": trace.trace_id,
                    "tool_count": len(tool_names),
                },
            )
            cases.append(case)
    return cases

## 클라이언트 초기화

Trace를 가져오기 위한 `ObservabilityClient`와 변환을 위한 `CloudWatchSessionMapper`를 생성합니다.

Mapper는 원시 AgentCore Observability span을 구조화된 Strands Eval 객체로 변환합니다:
- 각 상호작용을 재구성하기 위해 trace_id별로 span을 그룹화
- Tool 호출을 추출하고 결과와 매칭
- 사용자 프롬프트(첫 번째 메시지)와 agent 응답(최종 출력) 식별
- AgentInvocationSpan(전체 상호작용)과 ToolExecutionSpan(각 tool 사용)을 생성

In [ ]:
obs_client = ObservabilityClient(
    region_name=AWS_REGION,
    log_group=SOURCE_LOG_GROUP,
)
mapper = CloudWatchSessionMapper()

end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(hours=LOOKBACK_HOURS)
start_time_ms = int(start_time.timestamp() * 1000)
end_time_ms = int(end_time.timestamp() * 1000)

## 세션 처리

메인 evaluation 루프입니다. 각 세션에 대해:
1. AgentCore Observability에서 span 가져오기
2. Mapper를 사용하여 span을 Strands Eval Session 형식으로 변환
3. 세션의 각 trace에서 evaluation Case 생성
4. 모든 case에 대해 OutputEvaluator 실행
5. Tool을 사용한 case에 대해 TrajectoryEvaluator 실행
6. 대시보드 상관관계를 위해 원본 trace ID와 함께 모든 결과를 AgentCore Observability에 로깅

각 세션에 대한 진행 상황이 출력됩니다. 오류는 캡처되고 로깅되며 루프를 중단하지 않습니다.

In [ ]:
all_session_results = []
total_cases_evaluated = 0
total_logs_sent = 0
all_tools_used = set()

for session_idx, session_info in enumerate(sessions_to_process):
    session_id = session_info.session_id
    print(f"[{session_idx + 1}/{len(sessions_to_process)}] {session_id}")

    try:
        trace_data = obs_client.get_session_data(
            session_id=session_id,
            start_time_ms=start_time_ms,
            end_time_ms=end_time_ms,
            include_runtime_logs=False,
        )

        if not trace_data.spans:
            all_session_results.append({"session_id": session_id, "status": "skipped", "reason": "no_spans"})
            continue

        session = trace_data.to_session(mapper)
        cases = create_cases_from_session(session, session_id, MAX_CASES_PER_SESSION)

        if not cases:
            all_session_results.append({"session_id": session_id, "status": "skipped", "reason": "no_cases"})
            continue

        for case in cases:
            for tool in case.metadata.get("actual_trajectory", []):
                all_tools_used.add(tool)

        # Run Output Evaluator
        output_experiment = Experiment(cases=cases, evaluators=[OutputEvaluator(rubric=output_rubric)])
        output_results = output_experiment.run_evaluations(task_fn)
        output_report = output_results[0]

        output_logged = 0
        for i, case in enumerate(cases):
            if log_case_result_to_cloudwatch(case, OUTPUT_EVALUATOR_NAME, output_report.scores[i], output_report.reasons[i] if i < len(output_report.reasons) else ""):
                output_logged += 1

        # Run Trajectory Evaluator
        trajectory_cases = [c for c in cases if c.metadata.get("actual_trajectory")]
        trajectory_score = None
        trajectory_logged = 0

        if trajectory_cases:
            traj_eval_cases = [
                Case(name=c.name, input=c.input, expected_output=c.expected_output, session_id=c.session_id,
                     metadata={**c.metadata, "trajectory_for_eval": c.metadata.get("actual_trajectory", [])})
                for c in trajectory_cases
            ]
            trajectory_experiment = Experiment(
                cases=traj_eval_cases,
                evaluators=[TrajectoryEvaluator(rubric=trajectory_rubric, trajectory_description={"available_tools": list(all_tools_used)})]
            )
            trajectory_results = trajectory_experiment.run_evaluations(trajectory_task_fn)
            trajectory_report = trajectory_results[0]
            trajectory_score = trajectory_report.overall_score

            for i, case in enumerate(traj_eval_cases):
                if log_case_result_to_cloudwatch(case, TRAJECTORY_EVALUATOR_NAME, trajectory_report.scores[i], trajectory_report.reasons[i] if i < len(trajectory_report.reasons) else ""):
                    trajectory_logged += 1

        all_session_results.append({
            "session_id": session_id,
            "status": "completed",
            "case_count": len(cases),
            "output_score": output_report.overall_score,
            "trajectory_score": trajectory_score,
            "logs_sent": output_logged + trajectory_logged,
        })
        total_cases_evaluated += len(cases)
        total_logs_sent += output_logged + trajectory_logged

    except Exception as e:
        all_session_results.append({"session_id": session_id, "status": "error", "error": str(e)})

print(f"\nCompleted: {len([r for r in all_session_results if r['status'] == 'completed'])} sessions, {total_cases_evaluated} cases, {total_logs_sent} logs sent")

## 요약

완료율, 평가된 총 case 수, output 및 trajectory evaluator의 평균 점수를 포함하여 평가된 모든 세션의 집계 통계입니다.

In [ ]:
completed = [r for r in all_session_results if r.get("status") == "completed"]
output_scores = [r["output_score"] for r in completed if r.get("output_score") is not None]
trajectory_scores = [r["trajectory_score"] for r in completed if r.get("trajectory_score") is not None]

print(f"Sessions: {len(completed)}/{len(all_session_results)} completed")
print(f"Cases evaluated: {total_cases_evaluated}")
print(f"CloudWatch logs sent: {total_logs_sent}")

if output_scores:
    print(f"Output score: avg={sum(output_scores)/len(output_scores):.2f}, min={min(output_scores):.2f}, max={max(output_scores):.2f}")
if trajectory_scores:
    print(f"Trajectory score: avg={sum(trajectory_scores)/len(trajectory_scores):.2f}, min={min(trajectory_scores):.2f}, max={max(trajectory_scores):.2f}")

## 세션별 결과

Output 및 trajectory 점수를 보여주는 각 세션의 개별 결과입니다. "skipped"로 표시된 세션은 span이나 유효한 case가 없었습니다. "error"로 표시된 세션은 처리 중 예외가 발생했습니다.

In [ ]:
for i, r in enumerate(all_session_results):
    status = r.get("status", "unknown")
    if status == "completed":
        print(f"{i+1}. {r['session_id'][:20]}... output={r.get('output_score', 0):.2f} traj={r.get('trajectory_score') or '-'}")
    else:
        print(f"{i+1}. {r['session_id'][:20]}... {status}")

## 결과 내보내기

추가 분석이나 보고를 위해 evaluation 결과를 JSON으로 저장합니다. 내보내기에는 구성, 요약 통계 및 세션별 결과가 포함됩니다.

In [ ]:
import json

export_data = {
    "evaluation_time": datetime.now(timezone.utc).isoformat(),
    "config": {
        "source_log_group": SOURCE_LOG_GROUP,
        "eval_results_log_group": EVAL_RESULTS_LOG_GROUP,
        "output_evaluator": OUTPUT_EVALUATOR_NAME,
        "trajectory_evaluator": TRAJECTORY_EVALUATOR_NAME,
    },
    "summary": {
        "total_sessions": len(all_session_results),
        "completed_sessions": len(completed),
        "total_cases": total_cases_evaluated,
        "avg_output_score": sum(output_scores) / len(output_scores) if output_scores else None,
        "avg_trajectory_score": sum(trajectory_scores) / len(trajectory_scores) if trajectory_scores else None,
    },
    "session_results": all_session_results,
}

with open(RESULTS_JSON_PATH, "w") as f:
    json.dump(export_data, f, indent=2)

print(f"Exported to {RESULTS_JSON_PATH}")